In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import math
import torch
import gpytorch
import sys
import re
import statistics
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [2]:
import simple_gp

In [3]:
# df = pd.read_csv('data/DDG_Dataset.csv')
df = pd.read_csv('data/DDG_Dataset_mini.csv')
df = df[df['Temperature'] != "'-"]
ndatas = df.shape[0]
print(df.columns)
df

Index(['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB',
       'Temperature', 'pH', 'ΔΔG', 'Reference', 'A', 'R', 'N', 'D', 'C', 'Q',
       'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V',
       'ASP', 'PHE', 'GLN', 'LYS', 'ILE', 'TYR', 'GLY', 'ASN', 'ARG', 'LEU',
       'TRP', 'ALA', 'THR', 'VAL', 'HIS', 'CYS', 'GLU', 'MET', 'PRO', 'SER'],
      dtype='object')


,Rowid,ID,Protein Name,Mutation,Chain,PDB,Temperature,pH,ΔΔG,Reference,...,TRP,ALA,THR,VAL,HIS,CYS,GLU,MET,PRO,SER
0,1,1,Tryptophan synthase alpha chain,E49M,A,1WQ5,298.95,7.0,4.60,PMID: 378988,...,0.0,4.0,2.0,4.0,0.0,0.0,0.0,1.0,2.0,2.0
1,11,13,Cellular tumor antigen p53,M237I,A,1TUP,283.15,7.2,-3.18,PMID: 1203434,...,0.0,2.0,1.0,3.0,3.0,6.0,2.0,2.0,2.0,3.0
2,21,31,Ribonuclease HI,S68T,A,2RN2,298.15,5.5,0.10,PMID: 1317795,...,2.0,4.0,2.0,4.0,1.0,1.0,5.0,0.0,1.0,1.0
3,31,45,Ribonuclease HI,H62P,A,2RN2,298.15,5.5,0.74,PMID: 1328237,...,0.0,2.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0
4,41,59,Ribonuclease HI,K95G,A,2RN2,322.95,3.0,1.70,PMID: 1331044,...,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,8531,13279,Transcriptional repressor arc,K6A,A,1ARR,298.15,7.5,0.40,PMID: 7664079,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0
854,8541,13289,Transcriptional repressor arc,E17A,A,1ARR,298.15,7.5,-0.50,PMID: 7664079,...,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0
855,8551,13299,Transcriptional repressor arc,R23A,A,1ARR,298.15,7.5,-1.10,PMID: 7664079,...,1.0,1.0,0.0,4.0,0.0,0.0,4.0,0.0,1.0,1.0
856,8561,13309,Transcriptional repressor arc,N29A,A,1ARR,298.15,7.5,-1.60,PMID: 7664079,...,0.0,1.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,2.0


In [4]:
# df['Temperature'] = df['Temperature'].astype('float64')
df['Temperature'] = df['Temperature'].astype('float64')
# print(df['Temperature'].unique())
data_x = df.drop(columns=['Rowid', 'ID', 'Protein Name', 'Mutation', 'Chain', 'PDB', 'ΔΔG', 'Reference']).to_numpy()
data_y = df['ΔΔG'].to_numpy()

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=1234)
pred_y = np.zeros(ndatas)
lower_y = np.zeros(ndatas)
upper_y = np.zeros(ndatas)
for index, (train_indices, valid_indices) in enumerate(kf.split(range(ndatas))):
    # print(index)
    # print(train_indices)
    # print(valid_indices)
    train_x, test_x = data_x[train_indices], data_x[valid_indices] 
    train_y = data_y[train_indices]
    means, lower, upper = simple_gp.predict(train_x, train_y, test_x, 5)
    pred_y[valid_indices] = means
    lower_y[valid_indices] = lower
    upper_y[valid_indices] = upper

In [6]:
df['pred_ΔΔG'] = pred_y
df['lower_ΔΔG'] = lower_y
df['upper_ΔΔG'] = upper_y
# df.to_csv('data/pred_DDG_Dataset_gpytorch.csv')
df.to_csv('data/tmp.csv')